<a href="https://colab.research.google.com/github/ya-stack/PDF_to_text/blob/main/insurance_date_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd 'gdrive/My Drive/Colab Notebooks/pdf2text'

/content/gdrive/My Drive/Colab Notebooks/pdf2text


In [ ]:
pip install pdfminer

     |████████████████████████████████| 4.2MB 4.7MB/s 
     |████████████████████████████████| 13.7MB 310kB/s 
  Created wheel for pdfminer: filename=pdfminer-20191125-cp36-none-any.whl size=6140064 sha256=cc7b63505eb76b3af51c8a93c72d1eeab7e7d4bbdd80980f6676a38e5c622a5f
  Stored in directory: /root/.cache/pip/wheels/e1/00/af/720a55d74ba3615bb4709a3ded6dd71dc5370a586a0ff6f326
Successfully built pdfminer


In [ ]:
pip install datefinder

In [ ]:
import sys
import datetime
import datefinder
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
from io import StringIO

In [ ]:
def find_between_r( s, first, last ):
    try:
        start = s.rindex( first ) + len( first )
        end = s.rindex( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [ ]:
def pdfparser_new(data):

    fp = open(data, 'rb')
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams,
                           imagewriter = None)
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # Process each page contained in the document.

    for page in PDFPage.get_pages(fp, check_extractable=False):
        interpreter.process_page(page)
        data =  retstr.getvalue()

    #print (data)
    sub1 = 'Bajaj Allianz General Insurance Company Ltd'
    sub2 = 'Edelweiss	General	Insurance	Company	Limited'
    sub3 = 'Go Digit General Insurance Limited'
    sub4 = 'Acko General Insurance Ltd'
    sub5 = 'BHARTI AXA GENERAL INSURANCE COMPANY LTD'
    sub6 = 'HDFC ERGO General Insurance Company Limited'
    sub7 = 'IFFCO-TOKIO GENERAL INSURANCE CO.LTD'
    sub8 = 'Future Generali India Insurance Company Limited'
    sub9 = 'National Insurance Company Limited'
    sub11 = 'ICICI LOMBARD GENERAL INSURANCE CO LTD.'
    sub12 = 'The Oriental Insurance Company Limited'
    sub13 = 'Royal Sundaram General Insurance Co. Limited'
    sub14 = 'SHRIRAM GENERAL INSURANCE COMPANY LTD'
    sub15 = 'THE NEW INDIA ASSURANCE CO. LTD'
    sub16 = 'UNITED INDIA INSURANCE COMPANY LIMITED'
    sub17 = 'AIG'
    if sub1.lower() in data.lower():
        split_text = data.split('\n')
        indexes = [i for i,x in enumerate(split_text) if x == 'POLICY DETAILS']
        ind_val = indexes[0]
        policy_number = split_text[ind_val + 4]
        policy_from = split_text[ind_val + 16].split(' ')
        policy_upto = split_text[ind_val + 17].split(' ')
        #policy_n = split_text[1123].split('\n')
        print(policy_from)
        policy_detail = {'insurance_company': 'Bajaj Allianz General Insurance Company Ltd',
                         'policy_number': policy_number,
                         'policy_inception_date': datetime.datetime.strptime(policy_from[2] + ' ' + policy_from[3] + ':00', '%d-%b-%Y %H:%M:%S'),
                         'policy_expiry_date': datetime.datetime.strptime(policy_upto[2] + ' ' + '23:59:59', '%d-%b-%Y %H:%M:%S')}
    elif sub2.lower() in data.lower():
        split_text = data.split('\n')
        indexes = [i for i,x in enumerate(split_text) if x == 'Policy\tDetails']
        policy_n = split_text[indexes[0] - 2].split('\t')
        time_period = split_text[indexes[0] - 6].split('\t')
        policy_detail = {'insurance_company': 'Edelweiss General Insurance Company Limited',
                         'policy_number': policy_n[3],
                         'policy_inception_date': datetime.datetime.strptime(time_period[4] + ' ' + time_period[5] + time_period[6], '%d-%b-%Y %I:%M%p'),
                         'policy_expiry_date': datetime.datetime.strptime(time_period[8] + ' ' + time_period[9]  + time_period[10], '%d-%b-%Y %I:%M%p')}
    elif sub3.lower() in data.lower():
        split_text = data.split('\n')
        indexes = [i for i,x in enumerate(split_text) if x == 'Policy Number']
        policy_detail = {'insurance_company': 'Go Digit General Insurance Limited',
                         'policy_number': split_text[indexes[0] + 8],
                         'policy_inception_date': datetime.datetime.strptime(split_text[indexes[0] + 13] + ' ' + split_text[indexes[0] + 16], '%d-%b-%Y %H:%M:%S'),
                         'policy_expiry_date': datetime.datetime.strptime(split_text[indexes[0] + 14] + ' ' + split_text[indexes[0] + 17], '%d-%b-%Y %H:%M:%S')}
    elif sub4.lower() in data.lower():
        split_text = data.split('\n')
        indexes = [i for i,x in enumerate(split_text) if x == 'PREMIUM RECEIPT']
        time = split_text[indexes[0] + 13].split(' ')
        policy_detail = {'insurance_company': 'Acko General Insurance Ltd',
                         'policy_number': split_text[indexes[0] + 15],
                         'policy_inception_date': datetime.datetime.strptime(time[0]+'-'+time[1]+'-'+'20'+time[2]+' '+'00:00:00', '%d-%b-%Y %H:%M:%S'),
                         'policy_expiry_date': datetime.datetime.strptime(time[6]+'-'+time[7]+'-'+'20'+time[8]+ ' '+'23:59:59', '%d-%b-%Y %H:%M:%S')}
    elif sub5.lower() in data.lower():
        split_text = data.split('\n')
        indexes = [i for i,x in enumerate(split_text) if x == 'Policy Period']
        policy_number = split_text[indexes[0] + 4]
        time = split_text[indexes[0] + 5].split(' ')
        date1 = time[1].split(',')
        date2 = time[8].split(',')
        policy_detail = {'insurance_company': 'BHARTI AXA GENERAL INSURANCE COMPANY LTD',
                         'policy_number': policy_number,
                         'policy_inception_date':datetime.datetime.strptime(date1[0] + '-' + time[0]+'-'+time[2]+ ' '+'00:00:00', '%d-%B-%Y %H:%M:%S'),
                         'policy_expiry_date':datetime.datetime.strptime(date2[0] + '-' + time[7]+'-'+time[9]+ ' '+'23:59:59', '%d-%B-%Y %H:%M:%S')}
    elif sub6.lower() in data.lower():
        split_text = data.split(' -')
        policy_no = split_text[0]
        policy = re.findall(r'\d+', policy_no)
        date = list(datefinder.find_dates(split_text[2]))
        print(date)
        policy_detail = {'insurance_company': 'HDFC ERGO General Insurance Company Limited',
                         'policy_number': policy[0]+policy[1]+policy[2]+policy[3]+policy[4]}
    elif sub7.lower() in data.lower():
        split_text = data.split('\n')
        indexes = [i for i,x in enumerate(split_text) if x == 'Date of Issuance']
        policy_n = split_text[indexes[0]-1].split(' ')
        date1 = split_text[indexes[0] + 1].split('  ')
        date2 = split_text[indexes[0] + 10].split(' On ')
        policy_detail = {'insurance_company': 'IFFCO-TOKIO GENERAL INSURANCE CO.LTD',
                         'policy_number': policy_n[3],
                         'policy_inception_date':datetime.datetime.strptime(date1[6], '%d/%m/%Y %H:%M:%S'),
                         'policy_expiry_date':datetime.datetime.strptime(date2[1], '%d/%m/%Y %H:%M:%S')}
    elif sub8.lower() in data.lower():
        split_text = data.split('\n')
        indexes = [i for i,x in enumerate(split_text) if x == 'INSURED DETAILS']
        policy_n = split_text[indexes[0]-6].split(' ')
        indexes2 = [i for i,x in enumerate(split_text) if x == 'Period of Insurance']
        date1 = split_text[indexes2[0] + 2].split('  ')
        date2 = split_text[indexes2[0] + 4].split(' ')
        policy_detail = {'insurance_company': 'Future Generali India Insurance Company Limited',
                         'policy_number': policy_n[1],
                         'policy_inception_date':datetime.datetime.strptime(date1[1] + ' ' + '00:01:00', '%d/%m/%Y %H:%M:%S'),
                         'policy_expiry_date':datetime.datetime.strptime(date2[3]+ ' ' + '23:59:59', '%d/%m/%Y %H:%M:%S')}
    elif sub9.lower() in data.lower():
        split_text = data.split('\n')
        indexes = [i for i,x in enumerate(split_text) if x == 'पॉिलसी संखया Policy Number: ']
        res = [i for i in split_text if 'Policy Effective from' in i]
        date = res[0].split(' ')
        policy_detail = {'insurance_company': 'National Insurance Company Limited',
                         'policy_number': split_text[indexes[0]+1],
                         'policy_inception_date':datetime.datetime.strptime(date[13] + ' ' + date[10]+':00', '%d/%m/%Y %H:%M:%S'),
                         'policy_expiry_date':datetime.datetime.strptime(date[20]+ ' ' + '23:59:59', '%d/%m/%Y %H:%M:%S')}
    
    elif sub11.lower() in data.lower():
        split_text = data.split('\n')
        indexes = [i for i,x in enumerate(split_text) if x == 'Policy No. & Type']
        indexes1 = [i for i,x in enumerate(split_text) if x == 'Accounting Code of Service']
        print(split_text[indexes1[0]])
        date = split_text[indexes1[0]+15].split(' ')
        policy_detail = {'insurance_company':'ICICI LOMBARD GENERAL INSURANCE CO LTD',
                         'policy_number': split_text[indexes[0]+25],
                         'policy_inception_date':datetime.datetime.strptime(date[0] + ' ' + '00:01:00', '%d-%b-%Y %H:%M:%S'),
                         'policy_expiry_date':datetime.datetime.strptime(date[2]+ ' ' + '23:59:59', '%d-%b-%Y %H:%M:%S')
                         }
    elif sub12.lower() in data.lower():
        split_text = data.split('\n')
        indexes = [i for i, x in enumerate(split_text) if x == 'Policy No               :']
        indexes1 = [i for i, x in enumerate(split_text) if x == 'Period of Insurance  :']
        date = split_text[indexes1[0]+2].split(' ')
        policy_detail = {'insurance_company': 'The Oriental Insurance Company Limited',
                         'policy_number': split_text[indexes[0]+2],
                         'policy_inception_date':datetime.datetime.strptime(date[4] + ' ' + '00:00:01', '%d/%m/%Y %H:%M:%S'),
                         'policy_expiry_date':datetime.datetime.strptime(date[8]+ ' ' + '23:59:59', '%d/%m/%Y %H:%M:%S')
                         }
    elif sub13.lower() in data.lower():
        split_text = data.split('\n')
        indexes = [i for i, x in enumerate(split_text) if x == 'Certificate of Insurance and Policy No.']
        indexes1 = [i for i, x in enumerate(split_text) if x == 'Policy Period:']
        date = split_text[indexes1[0]+1].split(' ')
        policy_detail = {'insurance_company':'Royal Sundaram General Insurance Co. Limited',
                         'policy_number': split_text[indexes[0]+1],
                         'policy_inception_date':datetime.datetime.strptime(date[4] + ' ' + date[1], '%d/%m/%Y %H:%M:%S'),
                         'policy_expiry_date':datetime.datetime.strptime(date[9]+ ' ' + date[6], '%d/%m/%Y %H:%M:%S')
                         }
    elif sub14.lower() in data.lower():
        split_text = data.split('\n')
        indexes = [i for i, x in enumerate(split_text) if x == 'Policy No ']
        indexes1 = [i for i, x in enumerate(split_text) if x == 'Gross Premium:']
        date = split_text[indexes1[0]-1].split(' ')
        policy_detail = {'insurance_company':'SHRIRAM GENERAL INSURANCE COMPANY LTD',
                         'policy_number': split_text[indexes[0]+6],
                         'policy_inception_date':datetime.datetime.strptime(date[7] + ' ' + '00:00:01', '%d/%m/%Y %H:%M:%S'),
                         'policy_expiry_date':datetime.datetime.strptime(date[11]+ ' ' + '23:59:59', '%d/%m/%Y %H:%M:%S')
                         }
    elif sub15.lower() in data.lower():
        string = find_between_r( data, "Private Car Package PolicyPolicy Number", "Insured Details" )
        policy_n = string.split('Vehicle')
        date = string.split(' ')
        print(string)
        date_from = find_between_r(string, 'CoverFrom: ', 'To:')
        print(date_from)
        policy_detail = {'insurance_company': 'THE NEW INDIA ASSURANCE CO. LTD',
                         'policy_number': policy_n[0],
                         'policy_inception_date':datetime.datetime.strptime(date_from, '%d/%m/%Y %I:%M:%S %p'),
                         'policy_expiry_date':datetime.datetime.strptime(date[8]+ ' ' + '23:59:59', '%d/%m/%Y %H:%M:%S')
                         }
    elif sub16.lower() in data.lower():
        string = find_between_r(data, "Policy No.", "Registration No.")
        policy_n = string.split('\xa0')
        string1 = find_between_r(string, "InsuranceFrom", "Particulars of Vehicle Insured")
        date = string1.split(' ')
        policy_detail = {'insurance_company': 'UNITED INDIA INSURANCE COMPANY LIMITED',
                         'policy_number': policy_n[0],
                         'policy_inception_date':datetime.datetime.strptime(date[4] + ' '+'00:00:01', '%d/%m/%Y %H:%M:%S'),
                         'policy_expiry_date':datetime.datetime.strptime(date[8]+ ' ' + '23:59:59', '%d/%m/%Y %H:%M:%S')
                         }
    elif sub17.lower() in data.lower():
        data = data.replace(u'\xa0', u' ')
        string = find_between_r(data, "Policy Details : Policy Number : ", "Policy Period : From")
        string1 = find_between_r(data, "Policy Period : From", "Premium Paid : ")
        date = string1.split(' ')
        policy_detail = {'insurance_company': 'Tata AIG General Insurance Company LTD',
                         'policy_number': string,
                         'policy_inception_date':datetime.datetime.strptime(date[1] + ' '+'00:00:01', '%d/%m/%Y %H:%M:%S'),
                         'policy_expiry_date':datetime.datetime.strptime(date[5]+ ' ' + '23:59:59', '%d/%m/%Y %H:%M:%S')
                         }

    return data#, policy_detail

In [ ]:
#text, policy = pdfparser_new('Bajaj INsurance POlicy.pdf')
#text, policy = pdfparser_new('Edelwise Insurance Policy.pdf')
#text, policy = pdfparser_new('Go Digit Insurance Policy.pdf')
#text, policy = pdfparser_new('Acko Insurance Policy.pdf')
#text, policy = pdfparser_new('Bharti Axa Insurance Policy.pdf')
#text, policy = pdfparser_new('HDFC Insurance Policy.pdf') #### Do not run it
#text, policy = pdfparser_new('Iffco Tokio Insurance Policy.pdf')
#text, policy = pdfparser_new('Future Generali Insurance Policy.pdf')
#text, policy = pdfparser_new('National Insurance Policy.pdf')
#text = pdfparser_new('Kotak Insurance Policy.pdf')     #### Do not run it
#text, policy = pdfparser_new('ICICI Insurance Policy.pdf')
#text, policy = pdfparser_new('Oriental Insurance Policy.pdf')
#text, policy = pdfparser_new('Royal Sundaram Insurance Policy.pdf')
#text, policy = pdfparser_new('Shri Ram Insurance Policy.pdf')
#text, policy = pdfparser_new('The New India Insurance Policy.pdf')
#text, policy = pdfparser_new('United Insurance Policy.pdf')
#text, policy = pdfparser_new('TATA AIG Insurance Policy.pdf')
text = pdfparser_new('SBI Insurance Policy.pdf')

In [ ]:
import pprint
pprint.pprint(policy)

{'insurance_company': 'Tata AIG General Insurance Company LTD',
 'policy_expiry_date': datetime.datetime(2020, 2, 13, 23, 59, 59),
 'policy_inception_date': datetime.datetime(2019, 2, 14, 0, 0, 1),
 'policy_number': '0159432761 00'}


In [ ]:
len(text)

30211

In [ ]:
text

'\x0c\x0c\x0c'

In [ ]:
text.find('Policy Inception Date:', 1)

6042

In [ ]:
text[6070]

'1'

In [ ]:
text_split = text.split('\xa0')

In [ ]:
text_split

['Auto',
 'Secure',
 'Private',
 'Car',
 'Package',
 'PolicyName',
 ':',
 'MR',
 'GEETESH',
 'BHARJAddress',
 ':',
 'B',
 '11/GF',
 'B',
 'BLOCK',
 'SHIVAJI',
 'VIHARJANTA',
 'COLONYDELHI',
 '\xad',
 '110027DELHIDELHIINDIADate',
 ':',
 '14/02/2019Your',
 'Policy',
 'Details',
 ':',
 'Policy',
 'Number',
 ':',
 '0159432761',
 '00Policy',
 'Period',
 ':',
 'From',
 '14/02/2019',
 'to.',
 'Midnight',
 'Of',
 '13/02/2020',
 'Premium',
 'Paid',
 ':',
 '',
 '20,461.00Dear',
 'MR',
 'GEETESH',
 'BHARJ,Welcome',
 'to',
 'Tata',
 'AIG',
 'General',
 'Insurance',
 'Company',
 'Limited’s',
 'family',
 '&',
 'we',
 'thankyou',
 'for',
 'choosing',
 'our',
 'Auto',
 'Secure',
 'Private',
 'Car',
 'Package',
 'Policy',
 'for',
 'your',
 'vehicleinsurance.We',
 'are',
 'enclosing',
 'Policy',
 'schedule',
 'cum',
 'certificate',
 'of',
 'insurance',
 'of',
 'your',
 'vehicle.You',
 'are',
 'requested',
 'to',
 'visit',
 'our',
 'website',
 'www.tataaiginsurance.in',
 'for',
 'policywording.We',
 'wou

In [ ]:
indexes = [i for i,x in enumerate(text_split) if x == 'Accounting Code of Service']
indexes

[39]

In [ ]:
text_split[45]

'16-Apr-2020 to 15-Apr-2021'

In [ ]:
text_split[313].split(' ')

['January',
 '28,',
 '2020',
 '0.00',
 'to',
 'Midnight',
 'of',
 'January',
 '27,',
 '2021']

In [ ]:
res = [i for i in text_split if 'Policy Effective from' in i]

In [ ]:
res

['  पॉिलसी पभावी समय घंटे, को Policy Effective from 17:30 hours, on 01/07/2019 की अधरराित तक to midnight of 30/06/2020']

In [ ]:
text_split[770]

'11:18'

In [ ]:
text_split[370]

'18 Oct 19 (00:00 hrs) to 17 Oct 20 (23:59'

In [ ]:
policy_number = text_split[24].split('\t')

In [ ]:
policy_number

['Policy/Certificate', 'No.', '', '900047026']

In [ ]:
pip install pdf2image

In [ ]:
!ls

'Acko Insurance Policy.pdf'	   'HDFC Insurance Policy.pdf'
'Bajaj INsurance POlicy.pdf'	   'Iffco Tokio Insurance Policy.pdf'
'Bharti Axa Insurance Policy.pdf'   insurance_date_extractor.ipynb
'Edelwise Insurance Policy.pdf'     pdfquery.ipynb
'Go Digit Insurance Policy.pdf'


In [ ]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.10 [154 kB]
Fetched 154 kB in 1s (178 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.10_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.10) ...
Setting up poppler-utils (0.62.0-2ubuntu2.10) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
from pdf2image import convert_from_path, convert_from_bytes
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)
images = convert_from_path('SBI Insurance Policy.pdf')


In [ ]:
images

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1660x2337 at 0x7FE7505399E8>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1660x2337 at 0x7FE750539C88>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1660x2337 at 0x7FE75077D518>]

In [ ]:
!pip install pytesseract

  Created wheel for pytesseract: filename=pytesseract-0.3.4-py2.py3-none-any.whl size=13431 sha256=d7d5a4bba09ae19434734523cc08f4998ffa5f286b19b4a5086e325743c62ab7
  Stored in directory: /root/.cache/pip/wheels/63/2a/a0/7596d2e0a73cf0aeffd6f6170862c4e73f3763b7827e48691a
Successfully built pytesseract


In [ ]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 2s (2,585 kB/s)
debconf: unable to initi

In [ ]:
import pytesseract
#pytesseract.pytesseract.tesseract_cmd = r'/usr/local/bin/pytesseract'

In [ ]:
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image

from IPython.display import Image 
#pil_img = Image(filename='data/empire.jpg')
display(images[0])

In [ ]:
text = pytesseract.image_to_string(images[0])

In [ ]:
text

'Customer Care Number\n1800-22-1111 (MTNL/BSNL user)\n1800-102-1111 (for other users)\ncustomer.care@sbigeneral.in\n\n \n\nPRIVATE CAR CERTLIOAT. OF JNSURANGE CUM POLICY SCHEDULE\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n   \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\nImportant Note: 1) The Validity of this Certificate of Insurance cum Schedule is subject to realization of the premium cheque. 2) This Insurance Policy cover is valid subject to availabilty of Complete and Correct Registration\nNumber within 15 days from the Date of Commencement of this Policy. 3) This insurance policy cover is valid subject to Warranties, Terms and Conditions of the policy.\nHTM | jj 14012020_196468\nAEA - Palo Cette No :\nEM0675433491N SpeedPost sia :(0000000003780475-04\nBNPL A/c. No.: MR/DA-West/1573/17-18. Policy Issue Date 213/01/2020\nName © Mr. ASHOK KUMAR costomeD 70000000005589935\nGeographical Area “India\nAddress : , HOUSE NO-10 BLOCK-A, SECTOR 52, Noida Head Post Office, Noida,

In [ ]:
text.split('\n')

['Customer Care Number',
 '1800-22-1111 (MTNL/BSNL user)',
 '1800-102-1111 (for other users)',
 'customer.care@sbigeneral.in',
 '',
 ' ',
 '',
 'PRIVATE CAR CERTLIOAT. OF JNSURANGE CUM POLICY SCHEDULE',
 '',
 ' ',
 '',
 ' ',
 '',
 ' ',
 '',
 ' ',
 '',
 ' ',
 '',
 ' ',
 '',
 ' ',
 '',
 ' ',
 '',
 '   ',
 '',
 ' ',
 '',
 ' ',
 '',
 ' ',
 '',
 ' ',
 '',
 ' ',
 '',
 ' ',
 '',
 ' ',
 '',
 ' ',
 '',
 ' ',
 '',
 ' ',
 '',
 ' ',
 '',
 'Important Note: 1) The Validity of this Certificate of Insurance cum Schedule is subject to realization of the premium cheque. 2) This Insurance Policy cover is valid subject to availabilty of Complete and Correct Registration',
 'Number within 15 days from the Date of Commencement of this Policy. 3) This insurance policy cover is valid subject to Warranties, Terms and Conditions of the policy.',
 'HTM | jj 14012020_196468',
 'AEA - Palo Cette No :',
 'EM0675433491N SpeedPost sia :(0000000003780475-04',
 'BNPL A/c. No.: MR/DA-West/1573/17-18. Policy Issue Date 2